In [ ]:
import torch
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
import rasterio
import math
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from rasterio.enums import Resampling

In [ ]:
from tcd_pipeline.data.dataset import SingleImageGeoDataset

In [ ]:
test_image = "../data/5c15321f63d9810007f8b06f_10_00000.tif"

Geospatial data can be extremely large while machine learning models are only able to predict small inputs due to memory constraints. Even using a high-end graphics card with > 32 GB of VRAM, predicting larger than 2048 x 2048 px tiles is difficult. As a result, predicting over tiles is the preferred way to process large images.

We normally assume that CNNs are translation invariant. This means that a feature is fully visible, it should be predicted with similar confidence whether it's at the edge or the centre of the image. In practice this is only true in the centre of a tile, since at the edges the network sees blank data and has less context to make the prediction. To avoid these edge effects, images are tiled in an overlapping fashion with the overap roughly equal to one "receptive field" of the network. Empirically, 512 px gives a good result in most cases.

Our pipeline provides some simple utility functions to generate arbitrary resolution tiles from a source orthomosaic. Since the model is trained at 10 cm/px, we resample imagery to that resolution (nominally) though we also perform some augmentation at prediction-time.

Drone orthomosaics are often much higher quality, at 1-5cm/px so we have to resample. We provide two options:

 - Resize the image on the fly
 - Resample the image and store to disk
 
The first option is fast and works if you want to process an image once. If you think you're likely to experiment with different prediction settings, resampling as a one-time process might make more sense. Rescaling on the fly is also approximate and assumes that linear scaling is appropriate. You might find that you get different results if you compare both methods, but it shouldn't affect things much.
 
In order to plug in to machine learning pipelines, we provide a `SingleImageDataset` that takes a single image as an input and returns tile "samples". You can then iterate over this dataset sequentially to load all the tiles (e.g. you can directly pass it to a dataloader).

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=1024, overlap=512)

The arguments to the function are mostly self-explanatory:

- The input `test_image`, which can be a path to a file or an existing `rasterio.DatasetReader`
- We specify our desired Ground Sample Distance (GSD), here 0.6 metres
- Samples will be returned at the requested `tile_size` with some `overlap`

Overlap is important. Since we're cutting the image into pieces, we have to consider that there may be objects at the edge of the frame that might be missed. To get around this, we can overlap the tiles by a small amount. This problem also manifests as worse predictions near the edges of the image, since the model doesn't have the full context that it would at the centre. You need to account for this when you merge tiled predictions!

It's important that the tile size is large enough to capture enough context about the image, and ideally large enough that the model can take full advantage of its receptive field (around 512 px for ResNet50). This implies that a good rule of thumb for tile size is no smaller than a single receptive field. The overlap is a bit more complicated:

- It should be larger than the largest single "thing" that you want to detect, for example if a tree can be 50 metres wide, you'd want to make sure that your overlap is double that. Imagine the worst case where a tree is say 100 px wide and 10 px are off the edge of the image. We
- Ideally it should be a full receptive field, but practically half is OK (so 256 px is common).

Let's have a look at what these settings produce.

In [ ]:
ds.visualise(midpoints=True, edges=True)

The pipeline generates tiles that are centre-weighted. This should not significantly affect your results, but it does mean that the image is processed symmetrically.

Due to the way the image is indexed, the first tile is in the upper left (i.e. not at 0,0)

In [ ]:
ds.visualise([0], edges=True)

Tilling proceeds to the right and then down (here we skip tile i=2 for clarity). The tiler will automatically expand the overlap if it's possible to do so. For example for any requested overlap that is less than `(tile_size - (image_width/n_tiles))` we can expand the overlap without adding more computation.

This becomes relevant later on because you can discard a larger edge region from each (interior) tile which will give potentially better results:

In [ ]:
ds.visualise([0,1,3], edges=True)

To see the effect of symmetric sampling more, we can have a look at increasing the overlap to a very large value:

In [ ]:
SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=1024, overlap=700).visualise(midpoints=True, edges=True)

If we set zero overlap, then we simply grid the image:

In [ ]:
SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=1024, overlap=0).visualise(midpoints=True, edges=True)

What happens if you set a tile size that's very slightly smaller than the width or height of the image? The tiler will first determine how many tiles are needed given the provided overlap, for a 2048x2048 image with 2047x2047 tiles, we need at least 2 tiles in each direction. Again, the tiler will maximise the overlap for the minimum number of tiles required.

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=2047, overlap=0)
ds.visualise(midpoints=True, edges=True)

This assumed that we didn't want to do any resampling (the base image has a GSD of 0.1). What happens if we want to sample at 0.2m/px?

Note here we visualise the entire image and all tiles (i.e. one) - the tile locations here are pre-scaling.

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.2, tile_size=1024, overlap=256)
ds.visualise(midpoints=True, edges=True)

In [ ]:
#If instead we visualise a tile, we see it's the correct size of 1024x1024
    
ds.visualise_tile(0)

Resampling happens in the following order:

1. Image windows are generated based on the bounds of the image
2. The image windows are transformed from geographic extent to pixel values
3. This region is read from the image
4. We compute the scale factor from the source to the target GSD
5. The image is blurred by this scale factor and then resized

The tiler first establishes whether the output image size can be achieved with zero overlap, for example here we've got a 2048 px image at 0.1 m/px and we request a 1024 px tile at 0.2 m/px. The extent is the same, so no overlap is required.

As a general rule, the tiler _should_ produce a set of tiles that satisfy maximum overlap, subject to the provided minimum, with the minimum required number of tiles.

Be careful selecting very large overlaps. This will produce a very small stride, for example below only 20 px which will result in ~100 slices in each axis (2048 / (1020-1000)^2:

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=1020, overlap=1000)
print(len(ds))

Ok great. Let's look at some tiles:

In [ ]:
ds = SingleImageGeoDataset(test_image)

In [ ]:
ds.visualise_tile(0)

The tiler will automatically pad the image with nodata values. Following suggestions from the original UNet paper (see Figure 2), and () when we predict on this image, we will want to ignore some of the data at the edge because the predictive power is worse there for reasons we discussed earlier (lack of data at the edges). This is the highlighted (green) area in the plot below. Note that with more recent Transformer-based models, this logic may not be quite as simple - these models have a more complex receptive field and in theory suffer less from edge effects compared to CNNs.

In [ ]:
ds.visualise_tile(0, show_valid=True, valid_pad=128)

If you don't want to return fixed size tiles, you can use `pad=False` in the dataset, which will return rectangular images near the edges:

In [ ]:
ds = SingleImageGeoDataset(test_image, pad_if_needed=False)
ds.visualise_tile(0, show_valid=True)

How do we handle edge cases?

First, what happens if you suggest a tile equal to the image dimensions? No problem, we get one tile:

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=2048, overlap=256)
ds.visualise_tile(0)

Larger? Also no problem, your image will be embedded in the tile size if you like, but you should specify `clip_tiles=False`

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.1, tile_size=4096, overlap=256)
ds.visualise_tile(0)

This is also handled if you request a different GSD to the source image:

In [ ]:
ds = SingleImageGeoDataset(test_image, target_gsd=0.15, tile_size=1024, overlap=512)
ds.visualise(midpoints=True)

In [ ]:
ds.visualise_tile(0)